In [106]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score

import pickle
import tempfile

In [64]:
def remove_highly_correlated_features(df, threshold=0.9):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return to_drop

def remove_constant_features(df):
    return [column for column in df.columns if df[column].nunique() == 1]

def standarise_float_columns(df):
    for column in df.columns:
        if df[column].dtype == 'float64':
            df[column] = (df[column] - df[column].mean()) / df[column].std()
    return df

def normalise_int_columns(df):
    for column in df.columns:
        if df[column].dtype == 'int64':
            df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
    return df

In [68]:
df = pd.read_csv('ecom-user-churn-data.csv')
y = df['target_class']
X = df.drop(columns=['target_class'])

In [75]:
X_processed = X.drop(remove_highly_correlated_features(df, threshold=0.7), axis=1)
X_processed = X_processed.drop(['visitorid'], axis=1)
X_processed = X_processed.drop(remove_constant_features(X_processed), axis=1)
X_processed = standarise_float_columns(X_processed)
X_processed = normalise_int_columns(X_processed)

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.3, random_state=42)

In [105]:
clf = RandomForestClassifier(max_depth=10, random_state=42, n_estimators=100, class_weight='balanced')
clf.fit(X_train, y_train)
print(balanced_accuracy_score(y_train, clf.predict(X_train)))
print(balanced_accuracy_score(y_test, clf.predict(X_test)))

0.7362500259440399
0.6754106189816068


In [108]:
with tempfile.NamedTemporaryFile(suffix='.pkl', delete=False) as tmp:
    pickle.dump(clf, tmp)
    tmp_name = tmp.name